                                                                                MACHINE LEARNING

Lo primero que se debe de hacer es un centrado y escalado de los datos, eliminando previamente datos que no necesito para ML:

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
#Dataframe para predecir

games = pd.read_csv("data/games_clean.csv")

games_ml = games.iloc[:, -56:]
games_ml['Duration'] = games.gameDuration
columnas = games_ml.columns

#Mi variable a predecir (target)

winner_ml = games.iloc[:, 3]

In [3]:
# Ahora el primer paso es convertir todas las variables de campeones y spells a dummies, puesto que son texto

games_ml = pd.get_dummies(games_ml)

# Ahora que lo tengo tengo que hacer el centrado y escalado para evitar condicionar a la red con valores grandes

from sklearn.preprocessing import scale
games_ml_scaled = scale(games_ml, axis=0, with_mean=True, with_std=True, copy=True)

In [4]:
from sklearn.decomposition import PCA

In [5]:
pca = PCA().fit(games_ml_scaled) # Calculamos la varianza de las componentes principales

In [6]:
# Lo mostramos 

import matplotlib.pyplot as plt

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Número de componentes')
plt.ylabel('Varianza en porcentaje (%)')
plt.title('Varianza explicada')
plt.savefig("imagenes/PCA.png")
plt.show()

<Figure size 640x480 with 1 Axes>

Se puede ver que nos podemos quitar 500 dimensiones sin problemas siguiendo con una varianza bastante decente...

Lo dejaré en que con el 90% de explicación me vale

In [7]:
list = []

pca = PCA(0.9)

componentes_principales = pca.fit_transform(games_ml_scaled)

In [8]:
from sklearn.model_selection import train_test_split

for n in range(componentes_principales.shape[1]):
    list.append("PC {:.0f}".format(n))
    
dataframe_ml_09 = pd.DataFrame(data = componentes_principales, columns = list)
dataframe_ml_09.head(2)

x_train, x_test, y_train, y_test = train_test_split(dataframe_ml_09, games.winner, test_size=0.30, random_state=97)

Crearé también un PCA con 50% de variación para las grandes reducciones de dimensionalidad

In [9]:
list_2 = []

pca_low = PCA(0.5)

componentes_principales_low = pca_low.fit_transform(games_ml_scaled)

for n in range(componentes_principales_low.shape[1]):
    list_2.append("PC {:.0f}".format(n))
    
dataframe_ml_low = pd.DataFrame(data = componentes_principales_low, columns = list_2)

dataframe_ml_low.head(2)

x_train_low, x_test_low, y_train_low, y_test_low = train_test_split(dataframe_ml_low, games.winner, test_size=0.30, random_state=97)

Ahora crearé una matriz difusa a partir de los datos originales (tras el dummy), para mejorar la eficiencia computacional

In [10]:
print("La difusión de la matriz en tanto por uno es de: {:.3f}".format(games_ml.astype(bool).sum(axis=1).sum(axis=0)/(games_ml.shape[0]*games_ml.shape[1])))

La difusión de la matriz en tanto por uno es de: 0.017


Es decir, se podría decir que en la matriz prácticamente todos los elementos son cero

In [11]:
# Creacióm de la matriz difusa

from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(games_ml)
sparse_matrix

x_train_sparse, x_test_sparse, y_train_sparse, y_test_sparse = train_test_split(sparse_matrix, games.winner, test_size=0.30, random_state=97)

Ahora que tengo el centrado y escalado y el PCA, es momento de empezar con los algoritmos clasificadores y regresores:

                                                                
                                                                
                                                                1) Redes Neuronales - Perceptrón Multicapa

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

df_resultados = pd.DataFrame()

In [13]:
#lbfgs para datasets pequeños, converge mejor. adam para grandes, converge peor

# Solver lento pero mejor, alpha (decay) pequeño, seed en 1, activación por ReLU y 50 neuronas en la capa intermedia

red_multicapa_50 = MLPClassifier(solver='lbfgs', alpha=0.001, random_state=1, activation = 'relu', hidden_layer_sizes=(50,))
red_multicapa_50.fit(x_train, y_train)
prediccion_red_multicapa_50 = red_multicapa_50.predict(x_test)

In [14]:
print("50 NEURONAS")
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(red_multicapa_50.score(x_train, y_train)))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(red_multicapa_50.score(x_test, y_test)))

df_resultados['50 Neuronas_Train'] = red_multicapa_50.score(x_train, y_train)
df_resultados['50 Neuronas_Test'] = red_multicapa_50.score(x_test, y_test)

50 NEURONAS
La puntuación del entrenamiento en tanto por 1 es de: 1.000
La puntuación del test en tanto por 1 es de: 0.917


In [15]:
print("Matriz de confusión")
print(confusion_matrix(y_test,prediccion_red_multicapa_50))

Matriz de confusión
[[7188  598]
 [ 686 6975]]


In [16]:
print(classification_report(y_test,prediccion_red_multicapa_50))

             precision    recall  f1-score   support

          1       0.91      0.92      0.92      7786
          2       0.92      0.91      0.92      7661

avg / total       0.92      0.92      0.92     15447



Se está cayendo en overfitting. Reducción de las neuronas de la capa intermedia:

In [17]:
red_multicapa_20 = MLPClassifier(solver='lbfgs', alpha=0.001, random_state=1, activation = 'relu', hidden_layer_sizes=(20,))
red_multicapa_20.fit(x_train, y_train)
prediccion_red_multicapa_20 = red_multicapa_20.predict(x_test)

In [18]:
print("20 NEURONAS")
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(red_multicapa_20.score(x_train, y_train)))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(red_multicapa_20.score(x_test, y_test)))

df_resultados['20 Neuronas_Train'] = red_multicapa_20.score(x_train, y_train)
df_resultados['20 Neuronas_Test'] = red_multicapa_20.score(x_test, y_test)

20 NEURONAS
La puntuación del entrenamiento en tanto por 1 es de: 1.000
La puntuación del test en tanto por 1 es de: 0.914


In [19]:
print("Matriz de confusión")
print(confusion_matrix(y_test,prediccion_red_multicapa_20))

Matriz de confusión
[[7150  636]
 [ 698 6963]]


In [20]:
print(classification_report(y_test,prediccion_red_multicapa_20))

             precision    recall  f1-score   support

          1       0.91      0.92      0.91      7786
          2       0.92      0.91      0.91      7661

avg / total       0.91      0.91      0.91     15447



Sigue estando el entrenamiento demasiado alto, demasiado overfitting

In [21]:
red_multicapa_10 = MLPClassifier(solver='lbfgs', alpha=0.001, random_state=1, activation = 'relu', hidden_layer_sizes=(10,))
red_multicapa_10.fit(x_train, y_train)
prediccion_red_multicapa_10 = red_multicapa_10.predict(x_test)

In [22]:
print("10 NEURONAS")
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(red_multicapa_10.score(x_train, y_train)))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(red_multicapa_10.score(x_test, y_test)))

df_resultados['10 Neuronas_Train'] = red_multicapa_10.score(x_train, y_train)
df_resultados['10 Neuronas_Test'] = red_multicapa_10.score(x_test, y_test)

10 NEURONAS
La puntuación del entrenamiento en tanto por 1 es de: 0.997
La puntuación del test en tanto por 1 es de: 0.912


In [23]:
print("Matriz de confusión")
print(confusion_matrix(y_test,prediccion_red_multicapa_10))

Matriz de confusión
[[7113  673]
 [ 689 6972]]


In [24]:
print(classification_report(y_test,prediccion_red_multicapa_10))

             precision    recall  f1-score   support

          1       0.91      0.91      0.91      7786
          2       0.91      0.91      0.91      7661

avg / total       0.91      0.91      0.91     15447



In [25]:
red_multicapa_10_d01 = MLPClassifier(solver='adam', alpha=0.001, random_state=1, activation = 'relu', hidden_layer_sizes=(10,), learning_rate_init=0.1)
red_multicapa_10_d01.fit(x_train, y_train)
prediccion_red_multicapa_10_d01 = red_multicapa_10_d01.predict(x_test)

In [26]:
print("10 NEURONAS, DECAY 0.1")
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(red_multicapa_10_d01.score(x_train, y_train)))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(red_multicapa_10_d01.score(x_test, y_test)))

df_resultados['10 Neuronas-Decay 0.1_Train'] = red_multicapa_10_d01.score(x_train, y_train)
df_resultados['10 Neuronas-Decay 0.1_Test'] = red_multicapa_10_d01.score(x_test, y_test)

10 NEURONAS, DECAY 0.1
La puntuación del entrenamiento en tanto por 1 es de: 0.935
La puntuación del test en tanto por 1 es de: 0.886


In [27]:
print("Matriz de confusión")
print(confusion_matrix(y_test,prediccion_red_multicapa_10_d01))

Matriz de confusión
[[6558 1228]
 [ 537 7124]]


In [28]:
print(classification_report(y_test,prediccion_red_multicapa_10_d01))

             precision    recall  f1-score   support

          1       0.92      0.84      0.88      7786
          2       0.85      0.93      0.89      7661

avg / total       0.89      0.89      0.89     15447



                                                                            2) KNN - K Nearest Neighbors

KNN tiene un problema con la alta dimensionalidad, puesto que se basa en distancias euclídeas. Debido a esto, tengo que hacer una reducción de la dimensionalidad aún más grande a costa de sacrificar precisión

In [29]:
from sklearn.neighbors import KNeighborsClassifier

Como tenemos dos tipos de partidas (ganadas por T1 y ganadas por T2), vamos a clasificar en dos...

Para empezar, iré con K = 5, aunque seguramente acabe en overfitting y haya que subir

In [30]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(x_train_sparse, y_train_sparse)
prediccion_knn5 = knn.predict(x_test_sparse)

In [31]:
score_train_knn_5 = knn.score(x_train_sparse, y_train_sparse)
score_test_knn_5  = knn.score(x_test_sparse, y_test_sparse)

print("KNN con K = 5")
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(score_train_knn_5))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(score_test_knn_5))

KNN con K = 5
La puntuación del entrenamiento en tanto por 1 es de: 0.967
La puntuación del test en tanto por 1 es de: 0.948


In [32]:
df_resultados['knn5_train'] = score_train_knn_5
df_resultados['knn5_test'] = score_test_knn_5
print(confusion_matrix(y_test_sparse, prediccion_knn5))

[[7401  385]
 [ 425 7236]]


La teoría dice que el K perfecto suele ser sqrt(total). Veamos a ver

In [33]:
import math as m

knn_sqrt = KNeighborsClassifier(n_neighbors=m.trunc(m.sqrt(x_train.shape[0])), weights='uniform')
knn_sqrt.fit(x_train_sparse, y_train_sparse)
prediccion_knn_sqrt = knn_sqrt.predict(x_test_sparse)

In [34]:
score_train_knn_sqrt = knn_sqrt.score(x_train_sparse, y_train_sparse)
score_test_knn_sqrt  = knn_sqrt.score(x_test_sparse, y_test_sparse)

print("KNN con K = {:.0f}".format(m.sqrt(x_train.shape[0])))
print("La puntuación del entrenamiento en tanto por 1 es de: {:.3f}".format(score_train_knn_sqrt))
print("La puntuación del test en tanto por 1 es de: {:.3f}".format(score_test_knn_sqrt))

KNN con K = 124
La puntuación del entrenamiento en tanto por 1 es de: 0.928
La puntuación del test en tanto por 1 es de: 0.921


In [35]:
df_resultados['knn_sqrt_train'] = score_train_knn_sqrt
df_resultados['knn_sqrt_test'] = score_test_knn_sqrt
print(confusion_matrix(y_test_sparse, prediccion_knn_sqrt))

[[7307  479]
 [ 738 6923]]


Como se puede ver, en este caso la precisión es inferior con sqrt.